In [1]:
import pymc as pm
import numpy as np
import arviz as az

%load_ext lab_black
%load_ext watermark

# Meta-analysis via hierarchical models

This example demonstrates ...

Adapted from [unit 7: rats_nocentering.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit7/rats_nocentering.odc).

Data for x can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/rats_nocenteringx.txt), and Y [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/rats_nocenteringY.txt).

Associated lecture video: Unit 7 Lesson 5

In [3]:
y = np.loadtxt("../data/rats_nocenteringy.txt")
x = np.loadtxt("../data/rats_nocenteringx.txt")
x.shape, y.shape

((5,), (30, 5))

## Problem statement

This example is taken from Gelfand et al (1990), and concerns 30 young rats whose weights were measured weekly for five weeks.

In [4]:
with pm.Model() as m:
    mu_alpha = pm.Normal("mu_alpha", 0, tau=1e-6)
    tau_alpha = pm.Gamma("tau_alpha", 0.001, 0.001)
    mu_beta = pm.Normal("mu_beta", 0, tau=1e-6)
    tau_beta = pm.Gamma("tau_beta", 0.001, 0.001)

    tau = pm.Gamma("tau", 0.001, 0.001)
    sigma = pm.Deterministic("sigma", 1 / tau**0.5)

    alpha = pm.Normal("alpha", mu_alpha, tau=tau_alpha, shape=(30, 1))
    beta = pm.Normal("beta", mu_beta, tau=tau_beta, shape=(30, 1))

    mu = alpha + beta * x
    pm.Normal("y", mu, tau=tau, observed=y)

    trace = pm.sample(3000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu_alpha, tau_alpha, mu_beta, tau_beta, tau, alpha, beta]


Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 61 seconds.


In [5]:
az.summary(trace, var_names=["~alpha", "~beta"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu_alpha,106.555,2.298,102.345,110.896,0.023,0.017,9715.0,8861.0,1.0
mu_beta,6.187,0.105,5.983,6.379,0.001,0.001,10933.0,8863.0,1.0
tau_alpha,0.010,0.004,0.004,0.018,0.000,0.000,6847.0,7829.0,1.0
tau_beta,4.323,1.500,1.866,7.116,0.016,0.012,9683.0,8481.0,1.0
tau,0.027,0.004,0.020,0.035,0.000,0.000,7258.0,8444.0,1.0
sigma,6.149,0.461,5.301,7.015,0.005,0.004,7258.0,8444.0,1.0


In [6]:
%watermark -n -u -v -iv -p aesara,aeppl

Python implementation: CPython
Python version       : 3.10.4
IPython version      : 8.4.0

numpy: 1.22.4
pymc : 4.0.0
arviz: 0.12.1

